In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Base URL of the webpage you want to scrape, assuming it changes with page number
base_url = 'https://www.bayareamodern.com/san-francisco-real-estate/?pg={}'


# Headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

def fetch_data(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # raises exception when not a 2xx response
        if response.status_code == 200:
            return response.text
    except requests.HTTPError as e:
        print(f'HTTP error occurred: {e}')  # HTTP error
    except Exception as e:
        print(f'Other error occurred: {e}')  # Other errors
    return None

def normalize_key(key):
    """ Normalize keys by removing dots and converting to lower case. """
    return key.replace('.', '').lower().strip()

def parse_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    listings_data = []

    for listing in soup.select('.si-listing'):
        try:
            price = listing.select_one('.si-listing__price-main').text.strip()
            address_street = listing.select_one('.si-listing__title-main').text.strip()
            address_city_zip = listing.select_one('.si-listing__title-description').text.strip()
            
            info_labels = listing.select('.si-listing__info-label')
            info_values = listing.select('.si-listing__info-value span')
            # Apply normalization to each key
            details = {normalize_key(label.text): value.text.strip() for label, value in zip(info_labels, info_values)}
            
            beds = details.get('beds')
            baths = details.get('baths')
            sq_ft = details.get('sqft')  # Note the normalized key
            year_built = details.get('year built')

            listings_data.append({
                'price': price,
                'address': f'{address_street}, {address_city_zip}',
                'beds': beds,
                'baths': baths,
                'sq_ft': sq_ft,
                'year_built': year_built
            })
        except AttributeError as e:  # Handle missing fields or changes in website structure
            print(f'Missing data or error in data extraction: {e}')
    
    return listings_data


def save_to_csv(data, filename='listings.csv'):
    keys = data[0].keys() if data else []
    with open(filename, 'a', newline='', encoding='utf-8') as output_file:  # Change to 'a' to append to the file
        dict_writer = csv.DictWriter(output_file, keys)
        if output_file.tell() == 0:  # Write header only if file is empty
            dict_writer.writeheader()
        dict_writer.writerows(data)

# Main scraping function
def main(base_url):
    page_number = 1
    while True:
        url = base_url.format(page_number)
        print(f'Scraping page {page_number}: {url}')
        html_content = fetch_data(url)
        if html_content:
            soup = BeautifulSoup(html_content, 'html.parser')
            # Find the 'next' link and check if it is not disabled
            next_button = soup.select_one('a.js-next:not([aria-disabled="true"])')
            listings = parse_html(html_content)
            if listings:
                save_to_csv(listings)

            if not next_button:  # If there's no 'next' link or it is disabled, stop scraping
                print(f'No more pages to scrape. Stopped at page {page_number}.')
                break

            page_number += 1
            time.sleep(1)  # Sleep to prevent overwhelming the server
        else:
            print(f'Failed to retrieve data from {url}')
            break

    print('Data has been written to CSV')

# Run the main function
if __name__ == '__main__':
    main(base_url)





Scraping page 1: https://www.bayareamodern.com/san-francisco-real-estate/?pg=1
Scraping page 2: https://www.bayareamodern.com/san-francisco-real-estate/?pg=2
Scraping page 3: https://www.bayareamodern.com/san-francisco-real-estate/?pg=3
Scraping page 4: https://www.bayareamodern.com/san-francisco-real-estate/?pg=4
Scraping page 5: https://www.bayareamodern.com/san-francisco-real-estate/?pg=5
Scraping page 6: https://www.bayareamodern.com/san-francisco-real-estate/?pg=6
Scraping page 7: https://www.bayareamodern.com/san-francisco-real-estate/?pg=7
Scraping page 8: https://www.bayareamodern.com/san-francisco-real-estate/?pg=8
Scraping page 9: https://www.bayareamodern.com/san-francisco-real-estate/?pg=9
Scraping page 10: https://www.bayareamodern.com/san-francisco-real-estate/?pg=10
Scraping page 11: https://www.bayareamodern.com/san-francisco-real-estate/?pg=11
Scraping page 12: https://www.bayareamodern.com/san-francisco-real-estate/?pg=12
Scraping page 13: https://www.bayareamodern.co